In [58]:
import preprocessing as pp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
# import tensorflow_datasets as tfds

# Data Cleaning & One-Hot Encoding

In [2]:
df = pp.import_df()
df = pp.impute_missing_values(df)
df = pp.one_hot_encode(df)

## Creating Diff Columns

In [4]:
# all_cols = list(df.columns)
# max_cols = pp.find_columns(all_cols, 'max')
# min_cols = pp.find_columns(all_cols, 'min')

In [5]:
# cols_to_find_diff = [col[:-4] for col in max_cols]

In [6]:
# def find_diff(df, col_list):
#     """
#     """
#     new_df = df.copy(deep=True)
#     for col in col_list:
#         col_name = col + '_diff'
#         min_col_name = col + '_min'
#         max_col_name = col + '_max'
#         new_df[col_name] = new_df[max_col_name] - new_df[min_col_name]
#     return new_df

In [7]:
# df = find_diff(df, cols_to_find_diff)

## Identifying features with high correlation to target

In [8]:
x = df.corr()['hospital_death']
x = list(zip(list(x.index),list(x)))
above_5 = [i for i, j in x if abs(j) > 0.1]
below_5 = [i for i, j in x if abs(j) <= 0.1]

In [9]:
# above_5

## Train/Test Split and Scaling Features

In [11]:
X = df_upsampled.drop('hospital_death',axis=1)
y = df_upsampled['hospital_death']

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=2,stratify=y)

In [10]:
# Separate majority and minority classes
df_majority = df[df.hospital_death==0]
df_minority = df[df.hospital_death==1]

df_minority_upsampled = resample(df_minority, 
                                 replace=True,        # sample with replacement
                                 n_samples=83798,     # to match majority class
                                 random_state= 303)   # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

In [13]:
continuous_col = ['age', 'bmi', 'height', 'pre_icu_los_days', 'weight', 'map_apache', 'resprate_apache',
                  'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min', 'd1_resprate_max',
                  'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max', 'd1_sysbp_min', 
                  'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min', 'd1_temp_max', 'd1_temp_min', 
                  'h1_diasbp_max', 'h1_diasbp_min', 'h1_heartrate_max', 'h1_heartrate_min', 'h1_mbp_max', 
                  'h1_mbp_min', 'h1_resprate_max', 'h1_resprate_min', 'h1_spo2_max', 'h1_spo2_min', 
                  'h1_sysbp_max', 'h1_sysbp_min', 'd1_bun_max', 'd1_bun_min', 'd1_calcium_max', 
                  'd1_calcium_min', 'd1_creatinine_max', 'd1_creatinine_min', 'd1_glucose_max', 'd1_glucose_min',
                  'd1_hco3_max', 'd1_hco3_min', 'd1_hematocrit_max', 'd1_hematocrit_min', 'd1_platelets_max',
                  'd1_platelets_min', 'd1_potassium_max', 'd1_potassium_min', 'd1_sodium_max', 'd1_sodium_min',
                  'd1_wbc_max', 'd1_wbc_min', 'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob']

In [14]:
#continuous_col = [x for x in continuous_col if x not in below_5]

In [15]:
ss = StandardScaler()

In [16]:
x_train.loc[:,continuous_col] = ss.fit_transform(x_train[continuous_col])
x_test.loc[:,continuous_col] = ss.transform(x_test[continuous_col])

/Users/kathringo/opt/anaconda3/envs/wids/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/kathringo/opt/anaconda3/envs/wids/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [17]:
x_train = pd.DataFrame(x_train)
y_train = pd.Series(y_train)

In [18]:
pp.find_columns(x_train.columns, '%')

[]

# Modeling

## Logistic Regression

In [59]:
LogisticRegression(random_state=2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=2, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [60]:
clf = LogisticRegression(random_state=2, solver='lbfgs').fit(x_train, y_train)
y_train_hat = clf.predict(x_train)

/Users/kathringo/opt/anaconda3/envs/wids/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [61]:
accuracy_score(y_train, y_train_hat)

0.7905814612607774

In [62]:
f1_score(y_test, y_pred)

0.9792008436840871

In [63]:
print(classification_report(y_test,y_pred))
confusion_matrix(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      0.96      0.98     16760
           1       0.96      1.00      0.98     16760

    accuracy                           0.98     33520
   macro avg       0.98      0.98      0.98     33520
weighted avg       0.98      0.98      0.98     33520



array([[16097,   663],
       [   47, 16713]])

In [64]:
roc_auc_score(y_test, y_pred)

0.97881861575179

In [66]:
y_test_hat = clf.predict_proba(x_test)

In [68]:
accuracy_score(y_test, y_test_hat)

0.7909904534606206

In [69]:
f1_score(y_test, y_test_hat)

0.7875682231655549

In [70]:
print(classification_report(y_test, y_test_hat))
confusion_matrix(y_test, y_test_hat)

              precision    recall  f1-score   support

           0       0.78      0.81      0.79     16760
           1       0.80      0.77      0.79     16760

    accuracy                           0.79     33520
   macro avg       0.79      0.79      0.79     33520
weighted avg       0.79      0.79      0.79     33520



array([[13527,  3233],
       [ 3773, 12987]])

In [71]:
roc_auc_score(y_test, y_test_hat)

0.7909904534606206

## Tree-Based Methods
### Gradient Boosting Classifier

In [19]:
# GBC = GradientBoostingClassifier()
# GBC.fit(x_train, y_train)
# y_pred = GBC.predict(x_test)

In [20]:
# accuracy_score(y_test, y_pred)

In [21]:
# f1_score(y_test, y_pred)

In [22]:
# roc_auc_score(y_test,y_pred)

In [23]:
# print(classification_report(y_test,y_pred))

### RANDOM FOREST

In [24]:
rfc = RandomForestClassifier(max_depth=20, n_estimators=100, min_samples_leaf=2,min_samples_split=5)

In [25]:
rfc.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [26]:
y_pred = rfc.predict(x_test)

In [104]:
# y_pred = rfc.predict_proba(x_test)

In [27]:
accuracy_score(y_test, y_pred)

0.97881861575179

In [28]:
f1_score(y_test, y_pred)

0.9792008436840871

In [29]:
print(classification_report(y_test,y_pred))
confusion_matrix(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      0.96      0.98     16760
           1       0.96      1.00      0.98     16760

    accuracy                           0.98     33520
   macro avg       0.98      0.98      0.98     33520
weighted avg       0.98      0.98      0.98     33520



array([[16097,   663],
       [   47, 16713]])

In [30]:
roc_auc_score(y_test, y_pred)

0.97881861575179

## Neural Network

In [72]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train.values, y_train.values))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test.values,y_test.values))

In [73]:
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 120000

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [74]:
model = tf.keras.Sequential([
    
    tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l2(0.3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dense(16, kernel_regularizer=tf.keras.regularizers.l2(0.3), activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])

In [75]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
             metrics=['acc'])

In [76]:
model.fit(train_dataset,validation_data=test_dataset,epochs=50)

Train for 1048 steps, validate for 262 steps
Epoch 1/50
1048/1048 [==============================] - 4s 3ms/step - loss: 3.3651 - acc: 0.7755 - val_loss: 0.5788 - val_acc: 0.7086
Epoch 2/50
1048/1048 [==============================] - 3s 2ms/step - loss: 0.4849 - acc: 0.7871 - val_loss: 0.4947 - val_acc: 0.7717
Epoch 3/50
1048/1048 [==============================] - 3s 2ms/step - loss: 0.4647 - acc: 0.7895 - val_loss: 0.5016 - val_acc: 0.7710
Epoch 4/50
1048/1048 [==============================] - 3s 2ms/step - loss: 0.4587 - acc: 0.7907 - val_loss: 0.4685 - val_acc: 0.7841
Epoch 5/50
1048/1048 [==============================] - 3s 2ms/step - loss: 0.4582 - acc: 0.7905 - val_loss: 0.6562 - val_acc: 0.6833
Epoch 6/50
1048/1048 [==============================] - 3s 2ms/step - loss: 0.4563 - acc: 0.7912 - val_loss: 0.6315 - val_acc: 0.7344
Epoch 7/50
1048/1048 [==============================] - 3s 2ms/step - loss: 0.4557 - acc: 0.7916 - val_loss: 0.9027 - val_acc: 0.6131
Epoch 8/50
1048/1

In [77]:
y_pred = model.predict(test_dataset)

In [78]:
f1_score(y_test,np.round(y_pred))

0.6975822433610781

In [79]:
roc_auc_score(y_test,np.round(y_pred))

0.5675119331742243

In [80]:
print(classification_report(y_test,np.round(y_pred)))

              precision    recall  f1-score   support

           0       0.98      0.14      0.24     16760
           1       0.54      1.00      0.70     16760

    accuracy                           0.57     33520
   macro avg       0.76      0.57      0.47     33520
weighted avg       0.76      0.57      0.47     33520



In [81]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  14080     
_________________________________________________________________
batch_normalization (BatchNo multiple                  512       
_________________________________________________________________
dense_1 (Dense)              multiple                  8256      
_________________________________________________________________
batch_normalization_1 (Batch multiple                  256       
_________________________________________________________________
dense_2 (Dense)              multiple                  2080      
_________________________________________________________________
batch_normalization_2 (Batch multiple                  128       
_________________________________________________________________
dense_3 (Dense)              multiple                  3

## Predict Hospital Death on Out of Sample Data Set

In [91]:
test = pd.read_csv('unlabeled.csv')
test['id'] = test['encounter_id']
test.set_index('id',inplace=True)
test = pp.impute_missing_values(test)
test = pp.one_hot_encode(test)
test = test.drop('hospital_death',axis=1)
test.loc[:,continuous_col] = ss.transform(test[continuous_col])

In [92]:
test.shape

(39308, 109)

In [93]:
X.shape

(167596, 109)

In [105]:
# y_test_pred = rfc.predict(test)
# y_test_pred = clf.predict_proba(test)
y_test_pred = rfc.predict_proba(test)

In [106]:
y_test_pred[:, 1]

array([0.19290481, 0.11678718, 0.08400173, ..., 0.19049815, 0.01256751,
       0.27272247])

In [107]:
# list(test.index)

In [108]:
submission = pd.DataFrame({'encounter_id':list(test.index), 'hospital_death':y_test_pred[:, 1]})

In [109]:
submission.to_csv(path_or_buf = 'sub.csv', index=False)

In [36]:
unlabeled = pd.read_csv('unlabeled.csv')

In [37]:
unlabeled

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,2,127112,7,NaN,56.0,21.102439,0,Caucasian,M,180.3,Emergency Department,Accident & Emergency,1105,admit,Neuro ICU,0.104167,0,68.60,3.7,301.0,405.01,0,0.0,NaN,19.0,0.67,NaN,2.0,5.0,0.0,1.0,276.0,105.0,42.0,0.0,57.0,NaN,NaN,NaN,NaN,5.0,134.0,36.6,1121.1264,0.0,4.7,NaN,NaN,74.0,51.0,74.0,51.0,100.0,78.0,NaN,NaN,86.0,60.0,86.0,60.0,20.0,8.0,100.0,96.0,NaN,NaN,121.0,79.0,121.0,79.0,37.1,36.6,NaN,NaN,55.0,51.0,55.0,51.0,88.0,84.0,NaN,NaN,68.0,60.0,68.0,60.0,18.0,8.0,100.0,98.0,NaN,NaN,91.0,79.0,91.0,79.0,36.6,36.6,3.7,3.7,NaN,NaN,19.0,19.0,9.3,9.3,0.67,0.67,276.0,167.0,24.0,24.0,13.8,13.8,42.0,42.0,NaN,NaN,NaN,NaN,223.0,223.0,4.8,4.8,135.0,134.0,4.7,4.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.13,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic
1,5,130737,178,NaN,NaN,19.500595,1,Caucasian,F,145.0,Operating Room,Operating